In [330]:
import pandas as pd
from espnff import League
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

In [13]:
# connection
cred = pd.read_csv("/Users/patrick.chen/Desktop/espn_cred.csv")
league = League(749478,
                2018,
                espn_s2=cred['espn2'][0],
                swid=cred['swid'][0])

# Standings Table

In [311]:
team_name = []
standings =[]
for i in range(12):
    team = league.teams[i]
    team_name.append(team.team_name)
    standings.append(str(team.wins)+"-"+str(team.losses))

team_standings = pd.DataFrame({'team':team_name,'standings':standings})

# Scores Table

In [312]:
scores = []
team_name = []
opp = []
week= ['Week 1','Week 2','Week 3','Week 4','Week 5','Week 6','Week 7',
       'Week 8','Week 9','Week 10','Week 11','Week 12','Week 13']*12
for i in range(12):
    team = league.teams[i]
    opp = opp+team.schedule
    scores = scores + team.scores
    for i in range(13):
        team_name.append(team.team_name)

In [313]:
for i in [scores,team_name,opp,week]:
    print(len(i))

156
156
156
156


In [314]:
df = pd.DataFrame({'Team':team_name,
                   'Week':week,
                   'Score':scores,
                   'Opp':opp})

In [315]:
df['Opp'] = df.Opp.astype(str).str.replace('Team(', '', regex=False).str.strip(")")

In [316]:
df2 = df.copy().drop('Opp',axis=1)

In [317]:
df2.columns = ['Opp','Week','Opp_Score']

In [318]:
df3 = df.merge(df2,on=['Opp','Week'])

In [319]:
df3['margin'] = df3.Score-df3.Opp_Score

In [320]:
names = df3.Team.unique().tolist()

In [321]:
# create lists
lost_margin = []
win_margin = []
avg = []
std = []
for n in names:
    avg.append(df3[(df3.Team==n)].Score.mean())
    std.append(df3[(df3.Team==n)].Score.std())
    lost_margin.append(df3[(df3.Team==n)&(df3.margin<0)].margin.mean())
    win_margin.append(df3[(df3.Team==n)&(df3.margin>0)].margin.mean())

In [322]:
# create df
df4 = pd.DataFrame(data={'team':names,
                   'avg_score':avg,
                   'std':std,
                   'win_margin':win_margin,
                   'lost_margin':lost_margin
                  })

In [323]:
df4.head()

,team,avg_score,std,win_margin,lost_margin
0,Krispy Kareem,81.803077,51.910525,15.145000,-21.643333
1,FULL CHUBB,83.472308,55.644435,35.943333,-37.010000
2,âï¸GODS PLANâï¸,70.315385,45.396111,8.910000,-23.542500
3,Settlers of Saquon,92.515385,57.684160,30.890000,-34.090000
4,i am the kamara man,100.793846,60.428565,32.850000,-15.100000


In [324]:
df5 = df4.merge(team_standings, on ='team')

In [325]:
df5

,team,avg_score,std,win_margin,lost_margin,standings
0,Krispy Kareem,81.803077,51.910525,15.145000,-21.643333,4-6
1,FULL CHUBB,83.472308,55.644435,35.943333,-37.010000,6-4
2,âï¸GODS PLANâï¸,70.315385,45.396111,8.910000,-23.542500,2-8
3,Settlers of Saquon,92.515385,57.684160,30.890000,-34.090000,6-4
4,i am the kamara man,100.793846,60.428565,32.850000,-15.100000,6-4
5,Gurleys Gone Wild,82.749231,48.564979,16.557500,-11.750000,8-2
6,THE SUSPENDERS,85.930769,51.381399,17.825000,-14.960000,4-6
7,SMOOTH JAZZ,80.047692,47.417046,5.080000,-23.223333,4-6
8,TB on my Brown TDs,77.084615,45.311250,20.852000,-19.288000,5-5
9,WHITE PRIVILEGE,98.893846,59.327479,23.753333,-21.905000,6-4


In [326]:
df5.sort_values(['standings','avg_score'],ascending=False)

,team,avg_score,std,win_margin,lost_margin,standings
5,Gurleys Gone Wild,82.749231,48.564979,16.557500,-11.750000,8-2
4,i am the kamara man,100.793846,60.428565,32.850000,-15.100000,6-4
9,WHITE PRIVILEGE,98.893846,59.327479,23.753333,-21.905000,6-4
3,Settlers of Saquon,92.515385,57.684160,30.890000,-34.090000,6-4
10,RB Graveyard,84.573846,50.878180,16.333333,-13.860000,6-4
1,FULL CHUBB,83.472308,55.644435,35.943333,-37.010000,6-4
8,TB on my Brown TDs,77.084615,45.311250,20.852000,-19.288000,5-5
6,THE SUSPENDERS,85.930769,51.381399,17.825000,-14.960000,4-6
0,Krispy Kareem,81.803077,51.910525,15.145000,-21.643333,4-6
7,SMOOTH JAZZ,80.047692,47.417046,5.080000,-23.223333,4-6


In [327]:
columns = ['avg_score','std','win_margin','lost_margin']
titles = ['Average Score (mean)','BOOM or BUST (std)', 'Barely Won a Game (Win Margin)', 'Almost Won a Game (Loss Margin)']
standings = df5.standings.unique().tolist()

In [332]:
# create graph
for col,title in zip(columns,titles):
    df6 = df5.sort_values(col)

    # trace
    data = []
    for s in standings:
        trace = go.Bar(x=df6[df6.standings==s].team.tolist(),
                       y=df6[df6.standings==s][col].tolist(),name=s)
        data.append(trace)

    #layout
    layout = go.Layout(title=title,
                       barmode='group')

    # fig
    fig = go.Figure(data=data, layout=layout)
    
    py.iplot(fig)